# Chatbot in local instance Using RASA

Below codes are referenced from RASA documentation

For simplicity and understandable purpose just given single value

In [ ]:
domain.yml

intents:

# place your intents
 - greet
 - DigiT_turnaround_time
 - bye

entities:
# place your entities
 - month

templates:
# sample replies
 utter_greet:
  - "Hey, how can I help you?"
 utter_answer_time:
  - "unable to decide time in beforehand, usually we provide solution as soon as possible."
 utter_bye:
  - "Goodbye"

actions:
# templates (as they are reply actions),
# also custom actions if any
 - utter_greet
 - utter_answer_time



In [ ]:
stories.md

## story1              
* greet              
  - utter_greet
* DigiT_turnaround_time
  - utter_answer_time
* bye               
  - utter_bye


In [ ]:
nlu_train.md
'''
Let's consider we have our training data stored as json format, below is the single value from the data

{
"text": "I would like to digitalize my business based in India",
"intent": "DigiT",
"entities": [
{
"start": 31,
"end": 36,
"value": "India",
"entity": "location"
}
]
}

'''

In [ ]:

# Import the necessary packages from RASA

from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config
from rasa_nlu.model import Metadata, Interpreter


def train(data, config_file, model_dir):
    '''
    Functions to train model which takes three parameters 
    Data file, configuration file and directory where model gets saved
    '''
    train_data = load_data(data)                  # load the data using load_data()
    trainer = Trainer(RasaNLUConfig(config_file)) # Initialize Trainer class using RasaNlUconfig on configuration file
    trainer.train(train_data)                     # train the method on trainer with loaded data as parameter
    
    # Initialize model_directory 
    model_directory = trainer.persist(model_dir, fixed_model_name = 'cxbot')

# Training the model
# pass the training file, configuration file and the path to be stored

train('data/nlu_train.md', 'nlu_config.yml', 'models/nlu')

In [ ]:
# Test the model

# Initialize and load the model using RASA Interpreter object

interpreter = Interpreter.load('specify the path')

def test(message):
    print(interpreter.parse(message))
    
test('I would like to digitalize my business based in india')

# RASA Package Information

In [ ]:
# Train a dialog mode
# importing the necessary packages from RASA


import logging
from rasa_core import training
from rasa_core.actions import Action
from rasa_core.agent import Agent
from rasa_core.domain import Domain
from rasa_core.policies.keras_policy import KerasPolicy
from rasa_core.policies.memoization import MemoizationPolicy
from rasa_core.featurizers import MaxHistoryTrackerFeaturizer, BinarySingleStateFeaturizer
from rasa_core.channels.console import ConsoleInputChannel
from rasa_core.interpreter import RasaNLUInterpreter

# Function

def train_dialog(dialog_training_data_file, domain_file, path_to_model = 'models/dialogue'):
    logging.basicConfig(level='INFO')
    agent = Agent(domain_file,
              policies=[MemoizationPolicy(max_history=1)])
    training_data = agent.load_data(dialog_training_data_file)
    agent.train(
        training_data,
        augmentation_factor=50,
        epochs=200,
        batch_size=10,
        validation_split=0.2)
    agent.persist(path_to_model)

#train the dialog model
train_dialog('data/stories.md', 'domain.yml')

In [ ]:
# Loading Agent

rasaNLU = RasaNLUInterpreter("models/nlu/default/chat")
agent = Agent.load("models/dialogue", interpreter= rasaNLU)


# Below are the expected output

In [ ]:
agent.handle_message('Hi')

'Hey'

In [ ]:
agent.handle_message('How many months will it take to complete task')

'unable to decide time in beforehand, usually we provide solution as soon as possible.'
